In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import seaborn as sns #plot
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split 
from sklearn.linear_model import LinearRegression
from sklearn import metrics
import scipy.stats as stats

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
        
sns.set()        

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/competitive-data-science-predict-future-sales/sample_submission.csv
/kaggle/input/competitive-data-science-predict-future-sales/shops.csv
/kaggle/input/competitive-data-science-predict-future-sales/items.csv
/kaggle/input/competitive-data-science-predict-future-sales/item_categories.csv
/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv
/kaggle/input/competitive-data-science-predict-future-sales/test.csv


In [58]:
data = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/sales_train.csv')
test = pd.read_csv('/kaggle/input/competitive-data-science-predict-future-sales/test.csv')
test['idShopProd'] = test['shop_id'].astype(str)+test['item_id'].astype(str)
data['idShopProd'] = data['shop_id'].astype(str)+data['item_id'].astype(str)
df_new = pd.merge(test, data, on='idShopProd')

In [59]:
X_monthly = df_new.groupby(['shop_id_x','date_block_num'])['item_cnt_day'].sum()
tiendas_vivas = X_monthly[:,33].index
tiendas_vivas


Int64Index([ 2,  3,  4,  5,  6,  7, 10, 12, 14, 15, 16, 18, 19, 21, 22, 24, 25,
            26, 28, 31, 34, 35, 36, 37, 38, 39, 41, 42, 44, 45, 46, 47, 48, 49,
            50, 52, 53, 55, 56, 57, 58, 59],
           dtype='int64', name='shop_id_x')

In [60]:
filtered =df_new['shop_id_x'].map(lambda x: x in tiendas_vivas)
data_filtered= df_new[filtered]


In [61]:
X_monthly = data_filtered.groupby(['ID','date_block_num'])['item_cnt_day'].sum()
len(set(X_monthly.index.get_level_values(0)))

116132

In [ ]:
accuracy=0
prediction = []
for i in set(X_monthly.index.get_level_values(0)):
    X_full = X_monthly[i,].reindex(range(34),fill_value=0)
    X = X_full.index.values.reshape(-1,1)
    y = X_full.values.reshape(-1,1)
#    regressor = LinearRegression() 
#    regressor.fit(X, y)
#    prediction.append(regressor.predict([[34]])[0,0])    
# Splitting the dataset into the Training set and Test set
    x_train = X
    y_train = y.reshape(len(y))
    x_test = np.array([[34]])
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    x_train = sc.fit_transform(x_train)
    x_test = sc.transform(x_test)
    import lightgbm as lgb
    d_train = lgb.Dataset(x_train, label=y_train)
    params = {}
    params['learning_rate'] = 0.003
    params['boosting_type'] = 'gbdt'
    params['objective'] = 'binary'
    params['metric'] = 'binary_logloss'
    params['sub_feature'] = 0.5
    params['num_leaves'] = 10
    params['min_data'] = 50
    params['max_depth'] = 10
    clf = lgb.train(params, d_train, 100)
    #Prediction
    prediction.append(clf.predict(x_test))
#    break
    #Accuracy
#    from sklearn.metrics import mean_absolute_error
#    accuracy=mean_absolute_error(y_test,y_pred)+accuracy
    
#accuracy/len(set(X_monthly.index.get_level_values(0)))    

In [ ]:
pred = [float(x) for x in prediction]
pred

In [ ]:
output = pd.DataFrame({'item_cnt_month': pred},index = set(X_monthly.index.get_level_values(0)))
output = output.reindex(range(214200),fill_value = 0) 
output.insert(0,'Id',range(214200))
output.to_csv('submission.csv',index = False)

In [62]:
pd.read_csv('./submission.csv')

,Id,item_cnt_month
0,0,0.264706
1,1,0.000000
2,2,0.147059
3,3,0.029412
4,4,0.029412
...,...,...
214195,214195,0.147059
214196,214196,0.000000
214197,214197,0.176471
214198,214198,0.000000
